In [1]:
#importing the required modules
import os
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from scipy.stats import anderson, wilcoxon, ttest_ind

In [2]:
#reading the RepeatMasker output files into pandas dfs and set the genome parts (chromosomes and scaffolds) as indices
upstream_repeats=pd.read_csv('../results/upstream_repeats.tsv', sep='\t')
upstream_repeats=upstream_repeats.set_index('genoName')
downstream_repeats=pd.read_csv('../results/downstream_repeats.tsv', sep='\t')
downstream_repeats=downstream_repeats.set_index('genoName')
sample_repeats=pd.read_csv('../results/genomic_repeats.tsv', sep='\t')
sample_repeats=sample_repeats.set_index('genoName')

In [3]:
#get common repetitions in all three dataset (upstream, downstream, genome)
common_repeats = pd.Series(list(set(upstream_repeats['repName'].unique()) &
                             set(downstream_repeats['repName'].unique()) &
                             set(sample_repeats['repName'].unique())))


In [4]:
#get the number of common repetitions per chromosomes and scaffolds
def get_repeatnumber(chromosome, repname):
    upstream_subdf = upstream_repeats.loc[chromosome]
    downstream_subdf = downstream_repeats.loc[chromosome]
    sample_subdf = sample_repeats.loc[chromosome]
    return [list(upstream_subdf['repName']).count(repname),
           list(downstream_subdf['repName']).count(repname),
           list(sample_subdf['repName']).count(repname)]

In [5]:
#empty series for the repetitions
repeatnumbers = pd.Series(index = common_repeats, dtype = str)

In [6]:
#get the repeatnumber of every common repeats
for common_repeat in common_repeats:
    chromosomes = pd.Series(list(set(np.unique(sample_repeats.index.values))&
                           set(np.unique(upstream_repeats.index.values))&
                           set(np.unique(downstream_repeats.index.values))))
    samples = chromosomes.apply(get_repeatnumber, args = (common_repeat,))
    upstream_reps = []
    samples.apply(lambda sample : upstream_reps.append(sample[0]))
    downstream_reps = []
    samples.apply(lambda sample : downstream_reps.append(sample[1]))
    sample_reps = []
    samples.apply(lambda sample : sample_reps.append(sample[2]))
    repeatnumbers[common_repeat] = [upstream_reps, downstream_reps, sample_reps]

In [7]:
#function for the statistical analysis of repetitions
#upstream_repetitions = nested_list[0]
#downstream_repetitions = nested_list[1]
#sample_repetitions = nested_list[2]
def statistical_analysis(nested_list, which):
    sample1 = nested_list[which[0]]
    sample2 = nested_list[which[1]]
    norm1 = anderson(sample1)
    norm2 = anderson(sample2)
    stat1 = norm1[0]
    stat2 = norm2[0]
    critical_value1 = norm1[1][2]
    critical_value2 = norm2[1][2]
    if (stat1 > critical_value1) or (stat2 > critical_value2):
        try:
            significance = wilcoxon(sample1, sample2)[1]
            return significance
        except ValueError:
            pass
    else:
        try:
            significance = ttest_ind(sample1, sample2)[1]
            return significance
        except ValueError:
            pass

In [8]:
#statistics of upstream and downstream repetitions
upstream_downstream_stat = repeatnumbers.apply(statistical_analysis, args = ([0,1],))
#statistics of upstream and sample repetitions
upstream_sample_stat = repeatnumbers.apply(statistical_analysis, args = ([0,2],))
#statistics of downstream and sample repetitions
downstream_sample_stat = repeatnumbers.apply(statistical_analysis, args = ([1,2],))

/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/morestats.py:3155: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


In [9]:
#get the repeat category for every repeats
repeat_categories = pd.Series(repeatnumbers.index.values)
repeat_classes = repeat_categories.apply(lambda category : upstream_repeats.loc[upstream_repeats['repName'] == category]['repClass'].unique()[0])
repeat_classes.index = repeatnumbers.index.values

In [10]:
#invert repeat categories
repeats = pd.Series(repeat_classes.index.values)
repeats.index = repeat_classes

In [11]:
#get individual repeats for all repeat classes
repeat_families = pd.Series(repeat_classes.unique()).apply(lambda repeat_class:repeats[repeat_class].tolist())
repeat_families.index=repeat_classes.unique()

In [12]:
#get the new ordered header
def get_header(repeat_list):
    global header
    header += repeat_list

In [13]:
header = []
repeat_families.apply(get_header)

SINE              None
LINE              None
Low_complexity    None
Simple_repeat     None
tRNA              None
DNA               None
LTR               None
dtype: object

In [14]:
#create dataframe from the significance values
df = pd.DataFrame([upstream_downstream_stat,
upstream_sample_stat,
downstream_sample_stat])
df = df[header]
df.index = (['upstream_downstream',
'upstream_sample',
'downstream_sample'])

In [15]:
#function for drawing brackets to annotate each repeat family
def draw_brace(ax, xspan, text):
    """Draws an annotated brace on the axes."""
    xmin, xmax = xspan
    xspan = xmax - xmin
    ax_xmin, ax_xmax = ax.get_xlim()
    xax_span = ax_xmax - ax_xmin
    ymin, ymax = ax.get_ylim()
    yspan = ymax - ymin
    resolution = int(xspan/xax_span*100)*2+1 # guaranteed uneven
    beta = 300./xax_span # the higher this is, the smaller the radius

    x = np.linspace(xmin, xmax, resolution)
    x_half = x[:resolution//2+1]
    y_half_brace = (1/(1.+np.exp(-beta*(x_half-x_half[0])))
                    + 1/(1.+np.exp(-beta*(x_half-x_half[-1]))))
    y = np.concatenate((y_half_brace, y_half_brace[-2::-1]))
    y = ymin + (.15*y - .01)*yspan # adjust vertical position

    ax.autoscale(False)
    ax.plot(x, y, color='black', lw=1)

    ax.text((xmax+xmin)/2., ymin+.2*yspan, text, ha='center', va='bottom')

In [16]:
#create a function for the statistical annotation of the graph
def statistical_annotation(data, significance, positions, height, rounding):
    x1, x2 = positions[0],positions[1]
    maximum = max([max(data[0]),max(data[1])])
    y, h, col = maximum + height + 0.03, 0.03, 'k'
    plt.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=2.5, c = col)
    if significance < 0.05:
        plt.text((x1+x2)*.5, y+h, '$\it{P}$ ~ %s' % np.round(significance,rounding), ha='center',
                 va='bottom', color = col, fontsize = 20)
    else:
        plt.text((x1+x2)*.5, y+h, "n.s.", ha='center', va='bottom', color = col, fontsize = 20)

In [17]:
repeat_families

SINE               [CSINE1, MIRb, MIR3, CSINE2, MIRc, CSINE3A, MIR]
LINE              [L1MDa, L1A_Oc, L1MCa, L1MA6, L1MB7, L2b, L1M4...
Low_complexity         [AT_rich, GC_rich, GA-rich, A-rich, CT-rich]
Simple_repeat     [(CATATA)n, (TG)n, (CA)n, (TA)n, (CGTG)n, (T)n...
tRNA              [tRNA-Gln-CAA_, tRNA-Leu-TTA(m), tRNA-Ser-TCA(m)]
DNA                                        [Tigger7, MER5A1, MER5A]
LTR                 [ERVNOC_LTR, MLT1A1, MLT1A, LTR27_OC, LTR22_OC]
dtype: object

In [18]:
#visualizing
plt.style.use('fivethirtyeight')
bracket_start=0
fig, axes = plt.subplots(1, 1, figsize = (14,8))
upstream_downstream = axes.scatter(x = df.columns.values, y = df.loc['upstream_downstream'])
upstream_sample = axes.scatter(x = df.columns.values, y = df.loc['upstream_sample'])
downstream_sample = axes.scatter(x = df.columns.values, y = df.loc['downstream_sample'])
axes.plot(df.columns.values, (len(df.columns.values) * [0.05]), 'r',linewidth=0.75)
axes.legend((upstream_downstream, upstream_sample, downstream_sample),
          ("5' vs 3' flanking", "5' flanking vs genom",
           "3' flanking vs genom"),
           fontsize = 20,
           ncol = 3,
           loc = 'upper center')
axes.set_ylabel('$\it{P}$ value\n(inverted log scale)', fontsize = 20)
axes.set_xlabel('Repetitive elements', fontsize = 20)
axes.margins(x = 0.001)
axes.set_xticklabels(df.columns.values,rotation = 90)
axes.set_yscale('log')
axes.set_ylim([10**-3.1, 10**0.65])
axes.invert_yaxis()
for index, repeat_family in enumerate(repeat_families):
    bracket_end=(bracket_start+len(repeat_family))-1
    if repeat_families.index.values[index]=='Low_complexity':
        draw_brace(axes, (bracket_start,bracket_end),'Low\ncomplexity')
    elif repeat_families.index.values[index]=='Simple_repeat':
        draw_brace(axes, (bracket_start,bracket_end),'Simple\nrepeat')
    else:
        draw_brace(axes, (bracket_start,bracket_end),repeat_families.index.values[index])
    bracket_start+=len(repeat_family)
plt.tight_layout()
plt.savefig('../results/fig2.tiff',dpi=300)
plt.close()

/var/folders/g2/9pvgl5793mxgz8dg4mhwym0m0000gn/T/ipykernel_2150/3924923854.py:18: UserWarning: FixedFormatter should only be used together with FixedLocator
  axes.set_xticklabels(df.columns.values,rotation = 90)
